In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

import os
import cv2

from tqdm import tqdm
import random
from random import shuffle
import numpy as np         # dealing with arrays
# import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from matplotlib.image import imread
from os import makedirs
from os import listdir
from imutils import paths
from random import seed
from random import random
from sklearn.metrics import confusion_matrix
from random import random, shuffle
import pathlib

print(tf.__version__)

In [ ]:
#defining some variables
IMG_SIZE=64
ROWS = 256
COLS = 256
CHANNELS = 3

# set batch size
batch_size = 32

#set testing and training dataset 
Data_dir='Testing'
Test_DIR='Training'

#images for training
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  Data_dir,
  validation_split=0.2, 
  subset="training",
  seed=123,
  image_size=(ROWS, COLS),
  batch_size=batch_size)


#images for validation
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  Data_dir,
  validation_split=0.2, #20% of data for valifation
  subset="validation",
  seed=123,
  image_size=(ROWS, COLS),
  batch_size=batch_size)


In [ ]:
#display data with attached classname to ensure the dataset has loaded correctly
class_names = train_ds.class_names

plt.figure(figsize=(10, 10))

for images, labels in train_ds.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [46]:
num_classes = 4
epochs = 50

data_augmentation = keras.Sequential(
[
layers.experimental.preprocessing.RandomFlip("horizontal", 
input_shape=(ROWS, COLS, 3)),
layers.RandomRotation(0.2)
]
)
model = Sequential([
data_augmentation,
layers.experimental.preprocessing.Rescaling(1./255),
layers.Conv2D(32, 3, padding='same', activation='relu'),
layers.MaxPooling2D(),
layers.Dropout(0.2),
layers.Flatten(),
layers.Dense(128, activation='relu'),
layers.Dense(num_classes)])


In [47]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


# Step 5 Training

In [49]:
callback =tf.keras.callbacks.EarlyStopping(
  monitor='val_loss', min_delta=0.0000001, patience=10)

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
    callbacks=[callback]
)

Epoch 1/50


13/33 [==========>...................] - ETA: 21s - loss: 8.4981 - accuracy: 0.3221

KeyboardInterrupt: 

In [ ]:
model.save('saved_model')

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot( val_acc, label='Validation Accuracy')
plt.xticks(range(0,epochs)[0::2])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xticks(range(0,epochs)[0::2])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#images for Testing
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  Test_DIR,
  validation_split=0.0,
  seed=123,
  image_size=(ROWS, COLS),
  batch_size=batch_size)

model.evaluate(test_ds) 

In [52]:
y_test=[]
y_pred=[]
for images, labels in test_ds.take(10):
        predictions = model.predict(images)
        score = tf.nn.softmax(predictions)
        y_test.append(labels)
        y_pred.append(np.argmax(score,axis=1))
        
y_pred
y_test

NameError: name 'test_ds' is not defined

In [ ]:
shuffle(test_images)
for i in range(0, 10): # you can increase the range upto 50 to test, rigth now it is 10
    img = keras.preprocessing.image.load_img(
        test_images[i], target_size=(ROWS, COLS)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    plt.imshow(img_array[0,...].numpy().astype("uint8"))

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
   
    plt.show()